In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [3]:
from pathlib import Path

In [4]:
train_df_path = Path('/kaggle/input/nlp-getting-started/train.csv')
test_df_path = Path('/kaggle/input/nlp-getting-started/test.csv')
sample_submission_path = Path('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [80]:
train_df = pd.read_csv(train_df_path)
test_df = pd.read_csv(test_df_path)

In [6]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df.shape, test_df.shape

((7613, 5), (3263, 4))

In [8]:
train_df.isnull().mean()*100
test_df.isnull().mean()*100

id           0.000000
keyword      0.796813
location    33.864542
text         0.000000
dtype: float64

In [81]:
train_df = train_df.drop(columns=['location','id'],errors='ignore')
test_df = test_df.drop(columns=['location','id'], errors='ignore')

In [10]:
train_df.head()

,keyword,text,target
0,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [82]:
def fillnull(dataframe: pd.core.frame.DataFrame):
    dataframe = dataframe.fillna('NULL')
    return dataframe

In [83]:
train_df = fillnull(train_df)
test_df = fillnull(test_df)

In [13]:
#percentage of target labels
train_df['target'].value_counts(normalize=True) * 100

0    57.034021
1    42.965979
Name: target, dtype: float64

In [84]:
y = (train_df['target']).to_numpy()

In [92]:
X = train_df.drop(columns='target')

In [93]:
X = X['keyword'] + ". " +  X['text']

In [87]:
# X = (X.drop(columns=['keyword', 'text'], errors='ignore')).to_numpy()
X = (X.drop(columns=['keyword', 'text'], errors='ignore'))

In [98]:
X_test = test_df['keyword'] +  ". " +  test_df['text']

In [99]:
X_test

0                NULL. Just happened a terrible car crash
1       NULL. Heard about #earthquake is different cit...
2       NULL. there is a forest fire at spot pond, gee...
3          NULL. Apocalypse lighting. #Spokane #wildfires
4       NULL. Typhoon Soudelor kills 28 in China and T...
                              ...                        
3258    NULL. EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY...
3259    NULL. Storm in RI worse than last hurricane. M...
3260    NULL. Green Line derailment in Chicago http://...
3261    NULL. MEG issues Hazardous Weather Outlook (HW...
3262    NULL. #CityofCalgary has activated its Municip...
Length: 3263, dtype: object

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidfvect(data):
    vectorizer = TfidfVectorizer()
    vectorizer = vectorizer.fit(data)
    return vectorizer
vectorizer = tfidfvect(X)
X_vect = vectorizer.transform(X)
X_test_vect = vectorizer.transform(X_test)

In [108]:
X_vect.shape, X_test_vect.shape

((7613, 21668), (3263, 21668))

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
_X_vect = vectorizer.fit_transform(X)

In [95]:
_X_vect.shape

(7613, 21668)

In [24]:
#splitting data in train and validation set
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_vect, y,test_size = 0.2, random_state=42, stratify=y )

In [34]:
y_val.shape

(1523,)

In [54]:
#logistic regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [55]:
accuracy = logreg.score(X_val, y_val)
print("Accuracy: ", accuracy)

Accuracy:  0.8115561391989494


In [62]:
#knn
from sklearn.neighbors import KNeighborsClassifier
acc = []
for n_neighbors in range(1,1000, 5):
    knn = KNeighborsClassifier(n_neighbors)
    knn.fit(X_train, y_train)
    accuracy = knn.score(X_val, y_val)
    acc.append(accuracy)
    print("Accuracy: ", accuracy, "n:", n_neighbors)

Accuracy:  0.7537754432042022 n: 1
Accuracy:  0.768220617202889 n: 6
Accuracy:  0.7734734077478661 n: 11
Accuracy:  0.7636244254760342 n: 16
Accuracy:  0.7518056467498359 n: 21
Accuracy:  0.7524622455679579 n: 26
Accuracy:  0.7590282337491793 n: 31
Accuracy:  0.7760998030203545 n: 36
Accuracy:  0.7715036112934996 n: 41
Accuracy:  0.7793827971109653 n: 46
Accuracy:  0.788575180564675 n: 51
Accuracy:  0.7806959947472094 n: 56
Accuracy:  0.7839789888378201 n: 61
Accuracy:  0.7826657912015759 n: 66
Accuracy:  0.7872619829284307 n: 71
Accuracy:  0.7852921864740644 n: 76
Accuracy:  0.7944845699277742 n: 81
Accuracy:  0.7879185817465528 n: 86
Accuracy:  0.7846355876559422 n: 91
Accuracy:  0.7846355876559422 n: 96
Accuracy:  0.7852921864740644 n: 101
Accuracy:  0.7806959947472094 n: 106
Accuracy:  0.7774130006565988 n: 111
Accuracy:  0.7754432042022325 n: 116
Accuracy:  0.7774130006565988 n: 121
Accuracy:  0.7806959947472094 n: 126
Accuracy:  0.783322390019698 n: 131
Accuracy:  0.7806959947472

In [66]:
max(acc)

0.7944845699277742

In [69]:
knn = KNeighborsClassifier(n_neighbors=82)
knn.fit(X_train, y_train)
accuracy = knn.score(X_val, y_val)
acc.append(accuracy)
print("Accuracy: ", accuracy, "n:", n_neighbors)

Accuracy:  0.7951411687458962 n: 996


In [76]:
X_test_vect = vectorizer.fit_transform(X_test)

In [110]:
preds = knn.predict(X_test_vect)

In [114]:
submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [116]:
submission.drop(columns='target', inplace = True)

In [118]:
submission['target'] = preds

In [120]:
submission.to_csv('submission.csv')